# FIELDPROOF™ ML Prototype (Colab)
### Human Activity Recognition (HAR) → Sensor-Verified Task Execution

This notebook builds a **FIELDPROOF-style** prototype using a **public wearable sensor dataset** (UCI HAR).
We train several **course-level ML models**, measure results, and compare to a published baseline.

**Dataset:** UCI Human Activity Recognition Using Smartphones (30 subjects, 6 activities, 561 features, 10,299 windows)  
**Goal (course prototype):** Verify whether sensor signals match a required task (e.g., *Walking*) and classify activity states.


In [ ]:
!pip -q install pandas numpy scikit-learn matplotlib joblib
print('✅ Installed')

In [ ]:
import os, zipfile, urllib.request, pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

np.random.seed(42)
print("✅ Imports ready")


## 1) Download the public dataset (UCI HAR)

We download the official zip and load the provided train/test splits.

In [ ]:
DATA_DIR = pathlib.Path("data")
DATA_DIR.mkdir(exist_ok=True)

zip_url = "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"
zip_path = DATA_DIR / "uci_har.zip"
extract_dir = DATA_DIR / "UCI HAR Dataset"

if not zip_path.exists():
    print("Downloading dataset...")
    urllib.request.urlretrieve(zip_url, zip_path)
    print("✅ Downloaded:", zip_path)

if not extract_dir.exists():
    print("Extracting...")
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(DATA_DIR)
    print("✅ Extracted to:", extract_dir)

extract_dir


## 2) Load and prepare data

In [ ]:
def load_uci_har(base_dir: pathlib.Path):
    base = base_dir / "UCI HAR Dataset"
    X_train = pd.read_csv(base / "train" / "X_train.txt", sep=r"\s+", header=None)
    y_train = pd.read_csv(base / "train" / "y_train.txt", sep=r"\s+", header=None)[0]
    X_test  = pd.read_csv(base / "test" / "X_test.txt",  sep=r"\s+", header=None)
    y_test  = pd.read_csv(base / "test" / "y_test.txt",  sep=r"\s+", header=None)[0]

    act_map = pd.read_csv(base / "activity_labels.txt", sep=r"\s+", header=None, names=["id","label"])
    id_to_label = dict(zip(act_map["id"], act_map["label"]))

    return X_train, y_train.map(id_to_label), X_test, y_test.map(id_to_label), act_map

X_train, y_train, X_test, y_test, act_map = load_uci_har(DATA_DIR)
print("Train:", X_train.shape, "Test:", X_test.shape)
act_map


## 3) Class balance

In [ ]:
pd.Series(y_train).value_counts()

## 4) Train & compare models (course-level)

In [ ]:
models = {
    "LogReg": Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=2000))]),
    "KNN": Pipeline([("scaler", StandardScaler()), ("clf", KNeighborsClassifier(n_neighbors=15))]),
    "LinearSVM": Pipeline([("scaler", StandardScaler()), ("clf", LinearSVC())]),
    "RandomForest": RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
}

results = []
trained = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    results.append({
        "model": name,
        "accuracy": accuracy_score(y_test, pred),
        "f1_macro": f1_score(y_test, pred, average="macro")
    })
    trained[name] = model

results_df = pd.DataFrame(results).sort_values("accuracy", ascending=False).reset_index(drop=True)
results_df


## 5) Best model report + confusion matrix

In [ ]:
best_name = results_df.loc[0, "model"]
best_model = trained[best_name]
best_pred = best_model.predict(X_test)

print("Best model:", best_name)
print(classification_report(y_test, best_pred))

labels = list(act_map["label"])
cm = confusion_matrix(y_test, best_pred, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
cm_df


In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation="nearest")
plt.title(f"Confusion Matrix — {best_name}")
plt.xticks(range(len(labels)), labels, rotation=45, ha="right")
plt.yticks(range(len(labels)), labels)
plt.colorbar()
plt.tight_layout()
plt.show()


## 6) FIELDPROOF mapping: binary task verification (valid vs invalid)

Example: required task = **WALKING**.

In [ ]:
task_label = "WALKING"
y_train_bin = (y_train == task_label).astype(int)
y_test_bin  = (y_test == task_label).astype(int)

bin_model = Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=2000))])
bin_model.fit(X_train, y_train_bin)
bin_pred = bin_model.predict(X_test)

print("Binary verification for task:", task_label)
print("Accuracy:", round(accuracy_score(y_test_bin, bin_pred), 4))
print("F1:", round(f1_score(y_test_bin, bin_pred), 4))


## 7) Compare to published baseline (quick benchmark note)

Classic UCI HAR baselines often report ~**96%** multiclass accuracy with SVM.

In [ ]:
best_acc = float(results_df.loc[0, "accuracy"])
paper_reported_acc = 0.96  # benchmark value often cited for multiclass SVM on UCI HAR

print("Your best model:", best_name)
print("Your best accuracy:", round(best_acc, 4))
print("Published baseline (example):", paper_reported_acc)
print("Delta (yours - baseline):", round(best_acc - paper_reported_acc, 4))


## 8) Save best model (optional)

In [ ]:
import joblib
ART = pathlib.Path("artifacts")
ART.mkdir(exist_ok=True)
out = ART / f"best_model_{best_name}.joblib"
joblib.dump(best_model, out)
print("✅ Saved:", out)


## ✅ Done
- Public dataset loaded
- Multiple ML models compared
- FIELDPROOF-style verification demo included
- Ready for your course submission + write-up
